In [20]:
import string
import nltk
from nltk.corpus import stopwords
#nltk.download() # only need to download for once.

In [21]:
def clean_up_tweet(tweet):
    
    # remove punctuations.
    for p in string.punctuation:
        tweet = tweet.replace(p,'')
    
    tweet = tweet.split(' ')
    
    # remove stop words.
    stopWords = set(stopwords.words('english'))
    filtered = []
    for w in tweet:
        if w not in stopWords:
            filtered.append(w)
     
    # remove redundance.
    tweet_set = set(filtered)

    return tweet_set


In [22]:
# find the intersection of two tweets

def intersection(tweet1, tweet2):

    counter = 0
    for word1 in tweet1:
        for word2 in tweet2:
            if(word1 == word2):
                counter += 1
            
    return counter

In [27]:
def union(tweet1, tweet2):
    return len(tweet1) + len(tweet2) - intersection(tweet1,tweet2)

In [28]:
def jaccard_distance(tweet1, tweet2):
    return 1 - (intersection(tweet1, tweet2)/union(tweet1, tweet2))

In [29]:
t1 = 'it is weather is scary is is.'
t2 = 'the weather is nice is is!'

tweet1 = clean_up_tweet(t1)
tweet2 = clean_up_tweet(t2)


{'scary', 'weather'}
{'nice', 'weather'}
1
3
0.6666666666666667
